In [185]:
import sys
import pandas
sys.path.append('../')

from src.graph_utils import partition_graph_by_lonlat
import networkx as nx
from jurbey.jurbey import JURBEY

with open("../data/berlin.jurbey", 'rb') as tempf:
    g = JURBEY.load(tempf.read())
print(g.number_of_nodes())
g_partition = partition_graph_by_lonlat(g)


290100


**Convert to edge-based graph**

In [186]:
import networkx as nx
L = nx.line_graph(nx.DiGraph(g_partition))

In [187]:
nodes = list(L.nodes())
g_partition[nodes[10][0]][nodes[10][1]]['data']

Arc(arcType=<ArcType.LANE_STRAIGHT: 'LANE_STRAIGHT'>, roadClass=<RoadClass.MajorRoad: 2>, roadAccessibility=<RoadAccessibility.NoRestriction: 1>, metadata={'bicycle': 'no', 'highway': 'primary', 'lanes': '4', 'lit': 'yes', 'maxspeed': '50', 'name': 'Bismarckstraße', 'oneway': 'yes', 'postal_code': '10625', 'ref': 'B 2;B 5', 'surface': 'asphalt', 'turn:lanes': 'through|through|through;right|right'}, signs=[], vehicleAccessibility=[], geometry=[GeoCoordinates(lon=13.3207077, lat=52.5123944, alt=nan), GeoCoordinates(lon=13.3207877, lat=52.5123711, alt=nan)])

**Extract dynamic (speed) + static features from nodes**

In [247]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
enc = OneHotEncoder(handle_unknown='ignore')
ienc = OrdinalEncoder()
scaler = StandardScaler()
def arc_features(arc):
    arc = g_partition[arc[0]][arc[1]]
    return [
        arc['data'].metadata['highway'],
        arc['data'].metadata.get('surface', 'no_sur'),
        arc['data'].roadClass.name
    ],  [float(arc['data'].metadata.get('maxspeed', '50')), 
        int(arc['data'].metadata.get('lanes', '1'))]
def construct_features():
    data = list()
    data_ord = list()
    for node in L.nodes:
        data.append(arc_features(node)[0])
        data_ord.append(arc_features(node)[1])
    return enc.fit_transform(data), ienc.fit_transform(data_ord)
    
x, y = construct_features()
  

In [248]:
enc.categories_

[array(['access_ramp', 'corridor', 'living_street', 'platform', 'primary',
        'residential', 'secondary', 'secondary_link', 'service',
        'tertiary', 'tertiary_link', 'unclassified'], dtype=object),
 array(['asphalt', 'cobblestone', 'cobblestone:flattened', 'concrete',
        'concrete:plates', 'grass_paver', 'no_sur', 'paved',
        'paving_stones', 'sett'], dtype=object),
 array(['DirtRoad', 'LocalRoad', 'MajorRoad'], dtype=object)]

In [ ]:
ienc.categories_

In [250]:
x.shape

(6163, 25)

In [255]:
x

<6163x25 sparse matrix of type '<class 'numpy.float64'>'
	with 18489 stored elements in Compressed Sparse Row format>

**Preprocess adjacency matrix**

In [204]:
adj = nx.to_scipy_sparse_matrix(L, format="coo")
import scipy.sparse as sp
import numpy as np
import torch

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
                                    
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

# build symmetric adjacency matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
                                    
adj = normalize(adj + sp.eye(adj.shape[0]))
adj = sparse_mx_to_torch_sparse_tensor(adj)
                                    
                                    

In [205]:
adj.shape

torch.Size([6163, 6163])

In [206]:
#Our speed data uses segment ids, but the model uses sequential indexes, based on `.nodes()`
import math
id_to_idx = {}
# defaultdict won't do what you expect in Pandas
df = pandas.read_csv("../data/timeseries_speed_april_first_week.csv")
df = df.T
l = (df.isnull().mean() < 0.5).tolist()

indices = [i for i, x in enumerate(l) if x == True]
print(indices)

[4, 7, 16, 124, 157, 204, 351, 352, 379, 389, 430, 487, 492, 513, 539, 551, 568, 569, 587, 613, 630, 639, 649, 675, 680, 698, 711, 713, 780, 782, 784, 865, 1037, 1062, 1070, 1079, 1080, 1088, 1091, 1125, 1206, 1215, 1217, 1238, 1241, 1397, 1400, 1450, 1517, 1550, 1556, 1560, 1564, 1574, 1703, 1733, 1756, 1804, 1805, 1806, 1807, 1819, 1824, 1853, 1907, 1917, 1964, 1996, 1999, 2003, 2007, 2029, 2030, 2033, 2036, 2037, 2040, 2042, 2098, 2099, 2102, 2104, 2116, 2119, 2123, 2128, 2137, 2143, 2210, 2261, 2281, 2284, 2289, 2358, 2382, 2387, 2394, 2406, 2411, 2412, 2473, 2487, 2550, 2568, 2604, 2605, 2649, 2657, 2674, 2677, 2738, 2753, 2766, 2788, 2794, 2809, 2851, 2853, 2869, 2871, 2917, 2925, 2956, 3081, 3086, 3087, 3088, 3090, 3117, 3120, 3132, 3223, 3264, 3450, 3458, 3459, 3530, 3577, 3587, 3627, 3629, 3645, 3728, 3792, 3870, 3974, 3984, 3993, 4006, 4064, 4065, 4083, 4084, 4303, 4445, 4448, 4550, 4551, 4572, 4602, 4655, 4673, 4688, 4698, 4700, 4704, 4798, 4814, 4866, 4889, 4953, 4967, 4992

In [207]:
id_to_idx = {}

for idx, id_ in enumerate(L.nodes()):
    id_to_idx[id_] = idx
df = df.T
df = df.loc[:, df.columns != 'Unnamed: 0']

df2 = df['from_node']
df3 = df['to_node']

df_filled = df.loc[:, df.columns != 'from_node']
df_filled = df.loc[:, df.columns != 'to_node']


df_filled = df_filled.T
for column in df_filled:
    df_filled[column] = pandas.to_numeric(df_filled[column])

df_filled = df_filled.interpolate(method='nearest', axis=1)

df_filled = df_filled.fillna(method='backfill')
df_filled = df_filled.T
df_filled['from_node'] = df2
df_filled['to_node'] = df3

print(df_filled[0:10])


           0          1          2          3          4          5  \
0  10.854457  10.854457  10.854457  10.854457   5.680978   5.680978   
1  10.854457  10.854457  10.854457  10.854457   5.680978   5.680978   
2  10.854457  10.854457  10.854457  10.854457  10.486210  10.486210   
3  10.468811  10.468811  10.468811  10.468811  10.486210  10.486210   
4  10.468811  10.468811  10.468811  10.468811  10.636621  10.288534   
5  10.468811  10.468811  10.468811  10.468811  10.636621  10.288534   
6  10.854457  10.854457  10.854457  10.854457  10.831409   5.680978   
7  10.278846  10.854457  10.854457  10.854457  10.831409   5.680978   
8  10.278846  10.854457  10.854457  10.854457  10.831409  10.298252   
9  10.278846  10.854457  10.854457  10.854457  10.831409  10.298252   

           6          7          8          9  ...        136        137  \
0   7.036838   1.499997  10.088634  10.587459  ...  10.581123  10.850843   
1   7.036838   1.499997  10.088634  10.587459  ...  10.581123  10.

In [208]:
df[0:10]

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,from_node,to_node
0,NaN,NaN,NaN,10.8545,NaN,5.68098,7.03684,1.5,10.0886,10.5875,...,10.5811,NaN,NaN,NaN,NaN,NaN,10.8508,10.0364,628154368,1023689595
1,NaN,NaN,NaN,10.8545,NaN,5.68098,7.03684,1.5,10.0886,10.5875,...,10.5811,NaN,NaN,NaN,NaN,NaN,10.8508,10.0364,628154368,1023689595
2,NaN,NaN,NaN,NaN,NaN,10.4862,NaN,NaN,10.9372,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,27537239
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,26908815
4,NaN,NaN,NaN,10.4688,10.6366,10.2885,10.6175,1.75854,3.4044,10.2246,...,6.09877,10.2577,10.5862,NaN,10.6695,4.31342,10.3929,10.5183,628154370,3804638178
5,NaN,NaN,NaN,NaN,NaN,NaN,10.287,NaN,NaN,10.8336,...,10.5376,10.287,NaN,NaN,NaN,7.23855,10.498,10.4917,628154372,26938222
6,NaN,NaN,NaN,10.8545,NaN,5.68098,7.03684,1.5,5.04432,10.5875,...,0.961866,NaN,NaN,NaN,NaN,NaN,10.8508,10.0364,628154375,1560866145
7,10.2788,NaN,NaN,NaN,10.8314,NaN,3.14074,2.07388,0.385263,4.10448,...,10.5058,10.5173,NaN,NaN,NaN,10.1899,10.4367,4.19643,5791596551,1321327852
8,NaN,NaN,NaN,NaN,NaN,10.2983,1.22096,10.2983,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5791621141,4782446443
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5791621141,26875019


**Create rolling window tensor dataset**

In [259]:
import torch
import scipy.sparse
TOTAL_T_STEPS = 144

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
ienc = OrdinalEncoder()
    
def build_dataset_to_numpy_tensor(from_=0, to=TOTAL_T_STEPS, df=None):
    dataset = list()
    for t in range(from_, to):
        cat_features_at_t = [['primary', 'asphalt', 'MajorRoad']] * len(L.nodes)
        ord_features_at_t = [[50.0, 4]] * len(L.nodes)
        speed_features_at_t = [50] * len(L.nodes) 
        speed_is_nan_feature = [1] * len(L.nodes)
        for _, row in df.iterrows():

            arc = (row['from_node'], row['to_node'])
            cat_features_at_t[id_to_idx[arc]], ord_features_at_t[id_to_idx[arc]]  = arc_features(arc)
            speed_features_at_t[id_to_idx[arc]] = row[str(t)]
            if np.isnan(row[str(t)]): 
                speed_is_nan_feature[id_to_idx[arc]] = 0
        dataset.append(np.concatenate([scaler.fit_transform(np.array(speed_features_at_t).reshape(-1, 1)), 
                                       np.array(speed_is_nan_feature).reshape(-1, 1), 
                                       ienc.fit_transform(ord_features_at_t),
                                       enc.fit_transform(cat_features_at_t).toarray()], axis=1))
    return np.stack(dataset, axis=0)

Y = build_dataset_to_numpy_tensor(df=df)
Y_filled = build_dataset_to_numpy_tensor(df=df_filled)

In [260]:
Y.shape

(144, 6163, 29)

In [14]:
# serialize the sparse tensor dataset
# torch.save(X.indices(), "../data/dataset_st_indices.pt", pickle_protocol=4)
# torch.save(X.values(), "../data/dataset_st_values.pt", pickle_protocol=4)
# torch.save(X.size(), "../data/dataset_st_size.pt", pickle_protocol=4)

In [261]:
X = np.moveaxis(Y, source=(0,1,2), destination=(2,0,1))
X_filled = np.moveaxis(Y_filled, source=(0,1,2), destination=(2,0,1))


# num_vertices, num_features, num_timesteps
X.shape

(6163, 29, 144)

In [262]:
# Build mask tensor
X_masked = torch.where(torch.isnan(torch.from_numpy(X)), torch.tensor([0]), torch.tensor([1]))
X_masked = X_masked.bool()
print(X.shape)
print(X_masked.shape)

(6163, 29, 144)
torch.Size([6163, 29, 144])


In [263]:
def generate_dataset(X, num_timesteps_input, num_timesteps_output):
    """
    Takes node features for the graph and divides them into multiple samples
    along the time-axis by sliding a window of size (num_timesteps_input+
    num_timesteps_output) across it in steps of 1.
    :param X: Node features of shape (num_vertices, num_features,
    num_timesteps)
    :return:
        - Node features divided into multiple samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_input).
        - Node targets for the samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_output).
    """
    # Generate the beginning index and the ending index of a sample, which
    # contains (num_points_for_training + num_points_for_predicting) points
    indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
               in range(X.shape[2] - (
                num_timesteps_input + num_timesteps_output) + 1)]

    # Save samples
    features, target = [], []
    for i, j in indices:
        features.append(
            X[:, :, i: i + num_timesteps_input])
        target.append(X[:, :, i + num_timesteps_input: j])

    return torch.from_numpy(np.array(features)).permute(0,1,3,2), \
           torch.from_numpy(np.array(target)).permute(0,1,3,2)




def generate_dataset_concat(X, X_masked, num_timesteps_input, num_timesteps_output):
    """
    Takes node features for the graph and divides them into multiple samples
    along the time-axis by sliding a window of size (num_timesteps_input+
    num_timesteps_output) across it in steps of 1.
    :param X: Node features of shape (num_vertices, num_features,
    num_timesteps)
    :return:
        - Node data (features + labels) divided into multiple samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_input).
        
    """
    # Generate the beginning index and the ending index of a sample, which
    # contains (num_points_for_training + num_points_for_predicting) points
    indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
               in range(X.shape[2] - (
                num_timesteps_input + num_timesteps_output) + 1)]

    # Save samples
    features, target = [], []
    mask = []
    for i, j in indices:
        features.append(X[:, :, i: i + num_timesteps_input])
        target.append(X[:, 0, i + num_timesteps_input: j])
        mask.append(X_masked[:, 0, i + num_timesteps_input: j])

    return torch.from_numpy(np.array(features)), torch.from_numpy(np.array(target)),torch.stack(mask)

In [264]:
# training, validation, testing : 0.7, 0.1, 0.2
split_line1 = int(X.shape[2] * 0.7)
split_line2 = int(X.shape[2] * 0.9)
train_original_data = X_filled[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

train_mask = X_masked[:, :, :split_line1]
valid_mask = X_masked[:, :, split_line1:split_line2]
test_mask = X_masked[:, :, split_line2:]


look_back = 9
look_ahead = 1
# num_samples, num_nodes, num_timesteps, num_features

training_data, training_target, train_mask = generate_dataset_concat(train_original_data, train_mask,
                                                       num_timesteps_input=look_back,
                                                       num_timesteps_output=look_ahead)
valid_data, valid_target, valid_mask = generate_dataset_concat(val_original_data, valid_mask,
                                             num_timesteps_input=look_back,
                                             num_timesteps_output=look_ahead)
test_data, test_target, test_mask = generate_dataset_concat(test_original_data, test_mask,
                                               num_timesteps_input=look_back,
                                               num_timesteps_output=look_ahead)

print(f"shape of training: {training_data.shape}, {training_target.shape}, {train_mask.shape}")
print(f"shape of validation: {valid_data.shape}, {valid_target.shape}, {valid_mask.shape}")
print(f"shape of testing: {test_data.shape}, {test_target.shape}, {test_mask.shape}")

shape of training: torch.Size([91, 6163, 29, 9]), torch.Size([91, 6163, 1]), torch.Size([91, 6163, 1])
shape of validation: torch.Size([20, 6163, 29, 9]), torch.Size([20, 6163, 1]), torch.Size([20, 6163, 1])
shape of testing: torch.Size([6, 6163, 29, 9]), torch.Size([6, 6163, 1]), torch.Size([6, 6163, 1])


In [265]:
train_mask[1, :, :].shape

torch.Size([6163, 1])

In [267]:
import h5py
with h5py.File('data.hdf5', 'w') as f:
    f.create_dataset("train", data=training_data)
    f.create_dataset("valid", data=valid_data)
    f.create_dataset("test", data=test_data)
with h5py.File('target.hdf5', 'w') as f:
    f.create_dataset("train", data=training_target)
    f.create_dataset("valid", data=valid_target)
    f.create_dataset("test", data=test_target)
with h5py.File('mask.hdf5', 'w') as f:
    f.create_dataset("train", data=train_mask)
    f.create_dataset("valid", data=valid_mask)
    f.create_dataset("test", data=test_mask)

In [235]:
type(test_data)

torch.Tensor

In [233]:
data = {}
data['train'] = training_data
data['valid'] = valid_data
data['test'] = test_data

target['train'] = training_target
target['valid'] = valid_target
target['test'] = test_target

mask = {}
mask['train'] = train_mask
mask['valid'] = valid_mask
mask['test'] = test_mask
# batch shape: torch.Size([1, 6163, 26, 10])
print(valid_data.shape)
print(valid_mask[10, :, :])
print(valid_target[10, :, :])

print(valid_target[10, :, :].masked_select(valid_mask[0, :, :]))

torch.Size([20, 6163, 36, 9])
tensor([[ True],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])
tensor([[0.2421],
        [0.2421],
        [   nan],
        ...,
        [   nan],
        [   nan],
        [   nan]], dtype=torch.float64)
tensor([ 0.4326,  0.4197,  0.3346,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,  0.3471,  0.3471,     nan,
            nan,     nan,     nan,  0.3885,     nan,     nan,     nan,     nan,
         0.4559,  0.3665,  0.3665,     nan,     nan,  0.4451,  0.3479,     nan,
            nan,  0.3885,  0.3297,     nan,     nan,     nan,  0.3885,     nan,
            nan,     nan,     nan,  0.2709,  0.3003,  0.3585,  0.4485,  0.4485,
            nan,     nan,     nan,  0.3577,  0.3252,  0.3003,     nan,     nan,
         0.2708,  0.2708,     nan,     nan,     nan,     nan,     nan,  0.3227,
        -0.6791,  0.4455,     nan,     nan,     nan,     nan,     nan, 

**Now start training**

In [88]:
from test_tube import Experiment 
from pytorch_lightning import Trainer
import os

# PyTorch summarywriter with a few bells and whistles    
exp = Experiment(save_dir=os.getcwd())

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=30, train_percent_check=1)

gpu available: False, used: False


In [91]:
import imp  
import src.tgcn.temporal_spatial_model as l
import src.tgcn.layers.lstmcell as h
h = imp.reload(h)

l = imp.reload(l)
TGCN = l.TGCN
model = TGCN(input_dim=26, hidden_dim=26, layer_dim=2, output_dim=1, adj=adj, 
             datasets=data, mask=mask)
trainer.fit(model)

 59%|█████▊    | 65/111 [00:39<05:22,  7.01s/it, avg_val_loss=1.51, batch_nb=62, epoch=5, tng_loss=1.619, v_nb=29]

          Name        Type  Params
0      gc_lstm  GCLSTMCell    6318
1  gc_lstm.x2h      Linear    2808
2  gc_lstm.h2h      Linear    2808
3           fc      Linear      27
4      dropout     Dropout       0


 23%|██▎       | 26/111 [00:06<00:22,  3.79it/s, avg_val_loss=0.821, batch_nb=24, epoch=23, tng_loss=0.859, v_nb=29]]

KeyboardInterrupt: 